In [ ]:
mkdir .kaggle

In [ ]:
import json
token = {
  "username": "harsh777111raj",
  "key": "532bd9a20e4c1f82428b14be4d34a5ff"
}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!mv .kaggle /root/

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

cp: cannot stat '/content/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c quora-insincere-questions-classification -p /content 

 93% 51.0M/54.9M [00:01<00:00, 31.0MB/s]
100% 54.9M/54.9M [00:01<00:00, 51.5MB/s]
100% 5.94G/5.96G [01:27<00:00, 123MB/s]
100% 5.96G/5.96G [01:27<00:00, 72.9MB/s]
100% 4.09M/4.09M [00:00<00:00, 19.0MB/s]

100% 15.8M/15.8M [00:00<00:00, 29.8MB/s]



In [ ]:
!kaggle datasets download -d iezepov/gensim-embeddings-dataset -p /content 

100% 8.47G/8.47G [02:26<00:00, 70.1MB/s]
100% 8.47G/8.47G [02:26<00:00, 62.1MB/s]


In [ ]:
!unzip \*.zip

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  gensim-embeddings-dataset.zip
  inflating: GoogleNews-vectors-negative300.gensim  
  inflating: GoogleNews-vectors-negative300.gensim.vectors.npy  
  inflating: crawl-300d-2M.gensim    
  inflating: crawl-300d-2M.gensim.vectors.npy  
  inflating: glove.840B.300d.gensim  
  inflating: glove.840B.300d.gensim.vectors.npy  
  inflating: glove.twitter.27B.200d.gensim  
  inflating: glove.twitter.27B.200d.gensim.vectors.npy  
  inflating: numberbatch-en.gensim   
  inflating: numberbatch-en.gensim.vectors.npy  
  inflating: paragram_300_sl999.gensim  
  inflating: paragram_300_sl999.gensim.vectors.npy  

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.

In [ ]:
!unzip /content/test.csv.zip

Archive:  /content/test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
 !unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import pandas as pd
import numpy as np 
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")
train_df.shape

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer

def preprocessing(sentences):
  transformed = []
  lemmatizer = WordNetLemmatizer()
  puncts = [',','.', '"', ':', ')', '(', '-', '!', '?', '|', ';', 
          "'",  '&', '/', '[', ']', '>', '<', '%', '=', '#', '+', 
          '\\',  '§', '″', '′','¿','═','0','1','2','3','4','5','6','7','8','9','_','@','*','`']
  for sentence in sentences:
    sentence = str(sentence).lower()
    for punct in puncts:
      sentence = str(sentence).replace(punct, "")
    sentence = remove_stopwords(sentence)
    sentence = lemmatizer.lemmatize(sentence)
    transformed.append(sentence)
  return transformed

train_df.text = preprocessing(train_df.text)
train_df.selected_text = preprocessing(train_df.selected_text)
test_df.text = preprocessing(test_df.text)
train_df.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_features = 95000

tokenizer = Tokenizer(num_words = max_features, filters='')
tokenizer.fit_on_texts(list(train_df.question_text)+list(test_df.question_text))

X_train = tokenizer.texts_to_sequences(list(train_df.question_text))
X_test = tokenizer.texts_to_sequences(list(test_df.question_text))

vocabulary = tokenizer.word_index

y_train = train_df.target


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAXLEN = 70
X_train = pad_sequences(X_train, maxlen = MAXLEN, padding='pre')
X_test = pad_sequences(X_train, maxlen = MAXLEN, padding='pre')

In [ ]:
from gensim.models import KeyedVectors

embedding_glove = KeyedVectors.load(f"/content/glove.840B.300d.gensim")
embedding glove = KeyedVectors.load(f"GoogleNews-vectors-negative300/")
embedding_paragram = KeyedVectors.load(f"/content/paragram_300_sl999.gensim")

In [ ]:
def check_coverage(vocab, embedding_index, embedding_name):
    in_vocab = {}
    out_of_vocab = {}
    for word in vocab:
        try:
            in_vocab[word] = embedding_index[word]
        except:
            out_of_vocab[word] = vocab[word]
            
    print(f"{len(in_vocab)/len(vocab):.0%}"
         f"{embedding_name}")

check_coverage(vocabulary, embedding_glove, "glove")
check_coverage(vocabulary, embedding_google, "google")
check_coverage(vocabulary, embedding_paragram, "paragram")

56%glove
67%paragram


In [ ]:
def get_embedding_matrix(vocab, embedding_index):
    
    embedding_matrix = np.zeros((len(vocab) + 1, 300))

    for word, i in vocab.items(): 
        if word in embedding_index:
            embedding_matrix[i] = embedding_index[word]

    return embedding_matrix

embedding_matrix_glove = get_embedding_matrix(vocabulary, embedding_glove)
embedding_matrix_paragram = get_embedding_matrix(vocabulary, embedding_paragram)

embedding_matrix = np.mean([1.4 * embedding_matrix_glove, 
                             0.6 * embedding_matrix_paragram], 
                           axis = 0)


In [ ]:
embed_size = embedding_matrix.shape[1]

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, embed_size))
model.add(tf.keras.layers.Bidirectional(LSTM(embed_size, return_sequences = True)))
model.add(tf.keras.layers.SpatialDropout1D(0.2))
model.add(tf.keras.layers.Conv1D(128, 5))
model.add(tf.keras.layers.GlobalMaxPool1D(data_format='channels_last'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=1, verbose=2)
y_predicted = model.predict(X_test)